# **Part III - Transactions**

In [6]:
import requests
from pprint import pprint
import pandas as pd
import os
import dask 
from datetime import datetime, timedelta


# **Get data from API**

TO DO : 

- Run la cell en dessous 
- Avant, il faut aller [ici](https://docs.coinapi.io/), rentrer son adresse mail et clicker sur `get a free api key`
- Copier coller la clé dans un fichier dans le directory du notebook : nom du ficher --> `path_file_API`
- Aller dans le fichier `periods_API.txt`
- Choisir l'index (0,7,14,...) --> dans `index`

- Joao : Tu peux faire de 25 à 35 (des fois ça bug, donc va dans periods_API.txt, supprime les colonnes déjà Ok et retente avec une autre clé api).

In [50]:
def divide_time_periods() : 
    # Define the start and end dates
    start_date = datetime(2021, 1, 1)
    end_date = datetime(2021, 9, 1)

    # Generate a list of dates
    date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    datetime_strings = [dt.strftime('%Y-%m-%d') for dt in date_list]
    
    #Divide into slots of 7 (we can only make 10 calls per day)
    slot_size = 7
    date_slots = [datetime_strings[i:i+slot_size] for i in range(0, len(datetime_strings), slot_size)]
    
    with open("data/periods_API.txt", "w") as file : 
        for i in range(0, len(datetime_strings), slot_size) :
            data = str(i) + " : " + " ".join(datetime_strings[i:i+slot_size])
            file.write(data + "\n")

In [96]:
# Retrieve API key
def retrieve_api_key(name_file) : 
    api_key = None
    with open(name_file) as file:
        api_key = file.readline().strip()
    return api_key


# Retrieve historical data on Trades
def get_historical_trades(headers, index, day, save_dir) :
    
    parameters = {  
        "symbol_id": "BITSTAMP_SPOT_BTC_USD",
        "time_start" : day+'T00:00',
        "time_end" : day+'T23:59',
        "limit" : 100000, 
    }
    r = requests.get("https://rest.coinapi.io/v1/trades/BITSTAMP_SPOT_BTC_USD/history", 
                     headers=headers, params=parameters)

    # Check if the request was successful (status code 200)
    if r.status_code == 200:
        # Parse and print the trade data
        data = r.json()
        df = pd.DataFrame(data)
        
        #Save as csv 
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)

        name_file = str(index) + "_" + day
        df.to_parquet(save_dir+name_file, use_deprecated_int96_timestamps=True, compression="brotli")

    else:
        # Print an error message if the request was unsuccessful
        print(f"Error: {r.status_code} - {r.text}")


#To call every day 
def main_get_historical_trades(path_file_API, path_file_periods, index) : 
    api_key = retrieve_api_key(path_file_API)
    headers = {"X-CoinAPI-Key": api_key}

    days_list = None
    #Open the file and get the list of index to scrap (based on index)
    with open(path_file_periods) as file : 
        lines = file.readlines()
        for line in lines : 
            if line.split(":")[0].strip() == str(index) :
                days_list = line.split(":")[1][1:-1].split(" ")
    
    for day in days_list : 
        get_historical_trades(headers, index, day, save_dir = "data/raw/trades/")

#main_get_historical_trades("API_key_affolter.txt", "data/periods_API.txt", 28)
#main_get_historical_trades("API_key_ben.txt", "data/periods_API.txt", 28)
#main_get_historical_trades("API_key_epfl.txt", "data/periods_API.txt", 28)
#main_get_historical_trades("API_key_joanne.txt", "data/periods_API.txt", 28)
#main_get_historical_trades("API_key_reve.txt", "data/periods_API.txt", 28)
#main_get_historical_trades("API_key_data.txt", "data/periods_API.txt", 28)
#main_get_historical_trades("API_key_benProut.txt", "data/periods_API.txt", 35)
#main_get_historical_trades("API_key_anne.txt", "data/periods_API.txt", 35)
main_get_historical_trades("API_key_benjiplayer.txt", "data/periods_API.txt", 42)

In [94]:
pd.read_parquet("data/raw/trades/42_2021-02-15")

,symbol_id,time_exchange,time_coinapi,uuid,price,size,taker_side
0,BITSTAMP_SPOT_BTC_USD,2021-02-15T00:00:03.6900000Z,2021-02-15T00:00:03.8517555Z,230e0009-aa12-462c-b575-01027fb19e0e,48620.48,0.422000,SELL
1,BITSTAMP_SPOT_BTC_USD,2021-02-15T00:00:06.5500000Z,2021-02-15T00:00:06.6456087Z,7fdfd7fe-2d68-4bed-b939-ab7a871a57e0,48637.14,0.405330,BUY
2,BITSTAMP_SPOT_BTC_USD,2021-02-15T00:00:13.9310000Z,2021-02-15T00:00:13.9892542Z,e1eeada5-6c2a-4566-af3d-70d63f06cd87,48653.33,0.867467,BUY
3,BITSTAMP_SPOT_BTC_USD,2021-02-15T00:00:14.1070000Z,2021-02-15T00:00:14.1720409Z,e4d0f6ed-54d2-4af5-a0ea-3d77004d27fa,48675.00,0.010000,BUY
4,BITSTAMP_SPOT_BTC_USD,2021-02-15T00:00:14.5110000Z,2021-02-15T00:00:14.5848837Z,6dad05a5-8d8d-433c-8d72-d7459b8d14db,48674.12,0.422000,BUY
...,...,...,...,...,...,...,...
30961,BITSTAMP_SPOT_BTC_USD,2021-02-15T12:49:25.5240000Z,2021-02-15T12:49:25.6010705Z,d715e16c-b401-42be-9711-10b52f4e3c9c,47675.02,0.013109,SELL
30962,BITSTAMP_SPOT_BTC_USD,2021-02-15T12:49:27.0460000Z,2021-02-15T12:49:27.0966987Z,762979ae-b716-4d69-b12c-f89a8527697e,47675.03,0.063680,SELL
30963,BITSTAMP_SPOT_BTC_USD,2021-02-15T12:49:27.1850000Z,2021-02-15T12:49:27.2626353Z,0b108ca7-5738-43b6-8c80-8f53e10bf212,47659.76,0.044702,SELL
30964,BITSTAMP_SPOT_BTC_USD,2021-02-15T12:49:27.2320000Z,2021-02-15T12:49:27.2944981Z,9d3bb8b7-fb2c-4436-ba3a-dfcfb2bb03aa,47659.76,0.038215,SELL


- 08-01
- 11-01
- 29-01
- 